In [4]:
from pathlib import Path

from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

plot_code = """
import matplotlib.pyplot as plt

x = [1, 2, 3, 4]
y = [10, 20, 25, 30]

plt.plot(x, y)
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Sample Plot')
plt.savefig('sample_plot.png')
"""
async with DockerCommandLineCodeExecutor(image='autogen_image',work_dir=work_dir) as executor:  # type: ignore
    result = await executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code=plot_code),
        ],
        cancellation_token=CancellationToken(),
    )

In [5]:
result

CommandLineCodeResult(exit_code=0, output='', code_file='coding\\tmp_code_e514f31a1bb4587b3d19d2ba012ca9dcdc4384ec3cf0b55461232bf0f538575e.python')

In [1]:
import re
from dataclasses import dataclass
from typing import List

from autogen_core import DefaultTopicId, MessageContext, RoutedAgent, default_subscription, message_handler
from autogen_core.code_executor import CodeBlock, CodeExecutor
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)


@dataclass
class Message:
    content: str


@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._chat_history: List[LLMMessage] = [
            SystemMessage(
                content="""Write Python script in markdown block, and it will be executed.
Always save figures to file in the current directory. Do not use plt.show(). All code required to complete this task must be contained within a single response.""",
            )
        ]

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self._chat_history.append(UserMessage(content=message.content, source="user"))
        result = await self._model_client.create(self._chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        self._chat_history.append(AssistantMessage(content=result.content, source="assistant"))  # type: ignore
        await self.publish_message(Message(content=result.content), DefaultTopicId())  # type: ignore


def extract_markdown_code_blocks(markdown_text: str) -> List[CodeBlock]:
    pattern = re.compile(r"```(?:\s*([\w\+\-]+))?\n([\s\S]*?)```")
    matches = pattern.findall(markdown_text)
    code_blocks: List[CodeBlock] = []
    for match in matches:
        language = match[0].strip() if match[0] else ""
        code_content = match[1]
        code_blocks.append(CodeBlock(code=code_content, language=language))
    return code_blocks


@default_subscription
class Executor(RoutedAgent):
    def __init__(self, code_executor: CodeExecutor) -> None:
        super().__init__("An executor agent.")
        self._code_executor = code_executor

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        code_blocks = extract_markdown_code_blocks(message.content)
        if code_blocks:
            result = await self._code_executor.execute_code_blocks(
                code_blocks, cancellation_token=ctx.cancellation_token
            )
            print(f"\n{'-'*80}\nExecutor:\n{result.output}")
            await self.publish_message(Message(content=result.output), DefaultTopicId())


In [4]:
import tempfile

from autogen_core import SingleThreadedAgentRuntime
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient
from pathlib import Path

# work_dir = tempfile.mkdtemp()
work_dir = Path("coding")

# Create an local embedded runtime.
runtime = SingleThreadedAgentRuntime()

async with DockerCommandLineCodeExecutor(work_dir=work_dir, image='autogen_image') as executor:  # type: ignore[syntax]
    # Register the assistant and executor agents by providing
    # their agent types, the factory functions for creating instance and subscriptions.
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        # api_key="YOUR_API_KEY"
    )
    await Assistant.register(
        runtime,
        "assistant",
        lambda: Assistant(model_client=model_client),
    )
    await Executor.register(runtime, "executor", lambda: Executor(executor))

    # Start the runtime and publish a message to the assistant.
    runtime.start()
    await runtime.publish_message(
        Message("Create a plot of NVIDA vs TSLA stock returns YTD from 2024-01-01."), DefaultTopicId()
    )

    # Wait for the runtime to stop when idle.
    await runtime.stop_when_idle()
    # Close the connection to the model client.
    await model_client.close()



--------------------------------------------------------------------------------
Assistant:
```python
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Define the stock tickers for NVIDIA and Tesla
nvidia_ticker = 'NVDA'
tesla_ticker = 'TSLA'

# Set the start and end date
start_date = '2024-01-01'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Download the stock data using yfinance
nvidia_data = yf.download(nvidia_ticker, start=start_date, end=end_date)
tesla_data = yf.download(tesla_ticker, start=start_date, end=end_date)

# Calculate the daily returns
nvidia_data['Return'] = nvidia_data['Adj Close'].pct_change()
tesla_data['Return'] = tesla_data['Adj Close'].pct_change()

# Cumulative returns
nvidia_data['Cumulative Return'] = (1 + nvidia_data['Return']).cumprod()
tesla_data['Cumulative Return'] = (1 + tesla_data['Return']).cumprod()

# Plot the cumulative returns
plt.figure(figsize=(10, 6))
plt.plot(nvidia_data.index, nvidia_data['Cumulative Retu

In [9]:
import tempfile
import asyncio
from autogen_core import CancellationToken
from autogen_core.code_executor import with_requirements, CodeBlock
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
import pandas

@with_requirements(python_packages=["pandas"], global_imports=["pandas"])
def load_data() -> pandas.DataFrame:
    """Load some sample data.

    Returns:
        pandas.DataFrame: A DataFrame with sample data
    """
    data = {
        "name": ["John", "Anna", "Peter", "Linda"],
        "location": ["New York", "Paris", "Berlin", "London"],
        "age": [24, 13, 53, 33],
    }
    return pandas.DataFrame(data)

async def run_example():
    # The decorated function can be used in executed code
    with tempfile.TemporaryDirectory() as temp_dir:
        executor = LocalCommandLineCodeExecutor(work_dir=temp_dir, functions=[load_data])
        code = f"""from {executor.functions_module} import load_data

        # Use the imported function
        data = load_data()
        print(data['name'][0])"""

        result = await executor.execute_code_blocks(
            code_blocks=[CodeBlock(language="python", code=code)],
            cancellation_token=CancellationToken(),
        )
        print(result.output)  # Output: John

# Run the async example
await run_example()


C:\Users\chavv\AppData\Local\Temp\ipykernel_12940\416134394.py:25: UserWarning: The current event loop policy is not WindowsProactorEventLoopPolicy. This may cause issues with subprocesses. Try setting the event loop policy to WindowsProactorEventLoopPolicy. For example: `asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())`. See https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.ProactorEventLoop.
  executor = LocalCommandLineCodeExecutor(work_dir=temp_dir, functions=[load_data])


NotImplementedError: 